In [117]:
!pip install simpy plotly

In [118]:
import simpy

In [119]:
# very basic process

In [120]:
#

In [121]:
class User():
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.name = f"User-{id}"
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f"{self.name} starts task at %d" % self.env.now)
            task_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.process_task(task_duration))
            print(f"{self.name} ends task at %d" % self.env.now)

            # The charge process has finished and
            # we can start driving again.
            print(f"{self.name} starts verification at %d" % self.env.now)
            verification_duration = 2
            yield self.env.timeout(verification_duration)
            print(f"{self.name} ends verification at %d" % self.env.now)

    def process_task(self, duration):
        yield self.env.timeout(duration)


In [122]:
env = simpy.Environment()
user = User(1, env)
env.run(until=15)

User-1 starts task at 0
User-1 ends task at 5
User-1 starts verification at 5
User-1 ends verification at 7
User-1 starts task at 7
User-1 ends task at 12
User-1 starts verification at 12
User-1 ends verification at 14
User-1 starts task at 14


In [123]:
def user_generator(size, env):
    for i in range(size):
        User(i, env).action
        yield env.timeout(5)

In [124]:
env = simpy.Environment()
#user = User(env, res)
user_gen = env.process(user_generator(4, env))

env.run(until=15)

User-0 starts task at 0
User-1 starts task at 5
User-0 ends task at 5
User-0 starts verification at 5
User-0 ends verification at 7
User-0 starts task at 7
User-2 starts task at 10
User-1 ends task at 10
User-1 starts verification at 10
User-1 ends verification at 12
User-1 starts task at 12
User-0 ends task at 12
User-0 starts verification at 12
User-0 ends verification at 14
User-0 starts task at 14


In [125]:
# plotly example

In [126]:
!pip install plotly

In [127]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28'),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15'),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30')
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [128]:
import datetime
class Clock:
    def __init__(self):
        #self.base_epoch = datetime.datetime(2021,9,10,0,0).timestamp()
        self.base_epoch = datetime.datetime.now().timestamp()
        print(f"Clock created - base {self.base_epoch}")

    def to_date(self, tick):
        epoch_time = self.base_epoch + tick*60 #mn
        #print(f"Clock {epoch_time}")
        datetime_time = datetime.datetime.fromtimestamp(epoch_time)
        return datetime_time

In [129]:
clock = Clock()

Clock created - base 1631356937.664581


In [130]:
print(clock.to_date(1))
print(clock.to_date(2))

2021-09-11 12:43:17.664581
2021-09-11 12:44:17.664581


In [131]:
df = pd.DataFrame([
    dict(Task="Job A", Start=clock.to_date(1), Finish=clock.to_date(3)),
    dict(Task="Job B", Start=clock.to_date(2), Finish=clock.to_date(5)),
    dict(Task="Job C", Start=clock.to_date(3), Finish=clock.to_date(4))
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [132]:
class User():
    def __init__(self, id, env, monitor):
        self.id = id
        self.env = env
        self.monitor = monitor
        # ajoute apres sinon chevauchement
        self.taskid = 1
        self.name = f"User-{id}"
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = env.process(self.run())

    def run(self):
        while True:
            taskname = "process_task"
            self.taskid += 1
            self.monitor.report_start(self.name, taskname, self.taskid)
            print(f"{self.name} starts task at %d" % self.env.now)
            task_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.process_task(task_duration))
            print(f"{self.name} ends task at %d" % self.env.now)
            self.monitor.report_stop(self.name, taskname, self.taskid)
     
            # The charge process has finished and
            # we can start driving again.
            print(f"{self.name} starts verification at %d" % self.env.now)
            verification_duration = 2
            self.last_action_started_at = self.env.now
            yield self.env.timeout(verification_duration)
            print(f"{self.name} ends verification at %d" % self.env.now)

    def process_task(self, duration):
        yield self.env.timeout(duration)
        


In [133]:
import pandas as pd

In [138]:
class UsersMonitor:
    def __init__(self, env, clock):
        self.env = env
        self.start_data = []
        self.stop_data = []
        
    def report_start(self, username, taskname, taskid):
        mark = self.env.now
        self.start_data.append(
            dict(  
                StartMark=mark,
                Start=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )       
        
    def report_stop(self, username, taskname, taskid):
        mark = self.env.now
        self.stop_data.append(
            dict(  
                FinishMark=mark,
                Finish=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )
        
                
    def collect(self):
        df_start = pd.DataFrame(self.start_data)
        df_stop = pd.DataFrame(self.stop_data)
        df = pd.merge(df_start, df_stop, how='left', 
                      on = ['Username', 'Task', 'TaskId'])
        return df


In [139]:
user_monitor = UsersMonitor(env, clock)

In [140]:
user_monitor.report_start("U1", "process", "P1")
user_monitor.report_stop("U1", "process", "P1")
user_monitor.report_start("U2", "process", "P2")
print(user_monitor.collect())

                       Start  StartMark     Task TaskId Username  \
0 2021-09-11 12:57:17.664581         15  process     P1       U1   
1 2021-09-11 12:57:17.664581         15  process     P2       U2   

                      Finish  FinishMark  
0 2021-09-11 12:57:17.664581        15.0  
1                        NaT         NaN  


In [141]:
def user_generator(size, env, user_monitor):
    for i in range(size):
        User(i, env, user_monitor).action
        yield env.timeout(5)

In [142]:
# au passage suppression de magix numbers
sim_duration = 100
nb_users = 20

env = simpy.Environment()
user_monitor = UsersMonitor(env, clock)
#user = User(env, res)
# note un process est appele à chaque tick
user_gen = env.process(user_generator(nb_users, env, user_monitor))

env.run(until=sim_duration)

df_users = user_monitor.collect()
print("Users Monitor")
print(df_users)

User-0 starts task at 0
User-1 starts task at 5
User-0 ends task at 5
User-0 starts verification at 5
User-0 ends verification at 7
User-0 starts task at 7
User-2 starts task at 10
User-1 ends task at 10
User-1 starts verification at 10
User-1 ends verification at 12
User-1 starts task at 12
User-0 ends task at 12
User-0 starts verification at 12
User-0 ends verification at 14
User-0 starts task at 14
User-3 starts task at 15
User-2 ends task at 15
User-2 starts verification at 15
User-2 ends verification at 17
User-2 starts task at 17
User-1 ends task at 17
User-1 starts verification at 17
User-1 ends verification at 19
User-1 starts task at 19
User-0 ends task at 19
User-0 starts verification at 19
User-4 starts task at 20
User-3 ends task at 20
User-3 starts verification at 20
User-0 ends verification at 21
User-0 starts task at 21
User-3 ends verification at 22
User-3 starts task at 22
User-2 ends task at 22
User-2 starts verification at 22
User-2 ends verification at 24
User-2 sta

In [143]:
import plotly.express as px
import pandas as pd


In [144]:
fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()